In [81]:
import pandas as pd

data=pd.read_csv('commentsRedditVaccines.csv')
#this is to drop the bot comment 
data.drop(0,inplace=True)

print(data.loc[0:20]['comment_body'])

print("Before: ",len(data))
removed=data[(data['comment_body']=='[removed]')].index
deleted=data[(data['comment_body']=='[deleted]')].index
data.drop(removed,inplace=True)
data.drop(deleted,inplace=True)
data.drop_duplicates()
print("After: ",len(data))

1     Who's ahead in the pool for the third vaccine ...
2     "asymptomatic infection was reduced by 63 perc...
3     > and appears to prevent the spread of the vir...
4     The fact that there are multiple companies mak...
5     Wow! This is such great news, and makes me hop...
6     Scheduled to get my first round of the Moderna...
7                 INTO MY ASS, RIGHT UP IN THERE PLEASE
8     My 15th birthday is in March. I hope that by t...
9     So how many healthcare workers in the US can /...
10    As someone who worked on Moderna’s vaccine, I ...
11    If J&J gets approved in January, normal in the...
12    Any idea what kind of supply they have availab...
13    Did the Moderna vaccine trials also exclude pe...
14                     SHOVE IT IN ME YESTERDAY LETS GO
15                Go get your flu shot in the meantime.
16    My SIL is receiving this one next week. What i...
17                                Send bobs and vaccine
18    I’m going to sound dumb but is one vaccine

### **The columns are:** 
'post_id', 'post_author', 'post_date', 'post_title', 'post_score','post_permalink', 'post_url', 'comment_id','comment_author','comment_date', 'comment_parent_id', 'comment_edited', 'comment_score','comment_body'


In [197]:
#Removal of emojis, urls, and non-punctuation symbols like [ ]
import re
import emoji
from nltk.tokenize import word_tokenize
# nltk.download(stopwords)
from nltk.corpus import stopwords
stopWords=stopwords.words('english')
stopWords.extend(['.','?',',',"n't"," ' "])
def symbolCleaner(text):
    text=re.sub(r'\w+:\/{2}[\d\w-]+(\.[\d\w-]+)*(?:(?:\/[^\s/]*))*', '', text) #remove url
    text=re.sub(r'[^\w\s]','',text) #idk this regex tbh
    text=text.replace('\n',' ')
    text=text.replace('_',' ')
    text=text.replace('\t',' ')
    text=text.replace('\r',' ')
    text=text.replace('gt',' ')
    # https://stackoverflow.com/questions/33404752/removing-emojis-from-a-string-in-python
    emoji_pattern = re.compile(u"["
                        u"\U0001F600-\U0001F64F"  # emoticons
                        u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                        u"\U0001F680-\U0001F6FF"  # transport & map symbols
                        u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                        u"\U0001F1F2-\U0001F1F4"  # Macau flag
                        u"\U00002500-\U00002BEF"  # chinese char
                        u"\U0001F1E6-\U0001F1FF"  # flags
                        u"\U0001F600-\U0001F64F"
                        u"\U00002702-\U000027B0"
                        u"\U000024C2-\U0001F251"
                        u"\U0001f926-\U0001f937"
                        u"\U00010000-\U0010ffff"
                        u"\u2640-\u2642" 
                        u"\u2600-\u2B55"
                        u"\u200d"
                        u"\u23cf"
                        u"\u23e9"
                        u"\u231a"
                        u"\ufe0f"  # dingbats
                        u"\u3030"
                        u"\U0001F493"
                        u"\U0001F1F2"
                        u"\U0001F1F4"
                        u"\U0001F620"
                        u"\U0001F602"
                        "]+", re.UNICODE)
    text=emoji_pattern.sub(r'', text)
    text=emoji.get_emoji_regexp().sub(u'', text)
    text=re.sub('\s+',' ',text)
    return text

def removeStopwords(tokens):
    cleaned=[]
    for v in tokens:
            if v not in stopWords and len(v) > 0:
                cleaned.append(v)
    return cleaned

# Randomly selecting 10,000 samples from the corpus
randomSamples=data['comment_body'].sample(10000)
randomSamples[0:20]

16131    Yassssss pleaseeee inject me with that science...
18903    >There is no way to conclusively proof ANY new...
31689    It is a terrible plan, one fuck up and hospita...
18230              [relevant xkcd](https://xkcd.com/2391/)
16010    I know what you are saying. I trust the scienc...
3609     If only it could save us from the people who d...
10416    What was your personal reaction? As someone wh...
12552    Rushed? Do you think they started this from sc...
25481    They have even been able to do some testing ph...
33093    Yes, although most of us will probably just go...
30058    >The pandemic is ending\n\nLol, have you looke...
18284                 'Hol on...I'll be there in 21 hours.
11181                                 Congrats, Canada! :D
2757     Your comment has been removed because\n\n* **I...
4834                                Thank you and I shall.
1288     I'd think it not being approved here yet is wh...
13781                                     It is indeed..

### Using VADER to label the samples before processing them for further analysis

In [198]:
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
sid_obj=SentimentIntensityAnalyzer()

def sentimentLabel(data):#this creates another dictionary of sentiments and labels, yes im too lazy to just rewrite over the previous dictionary cause we're in a hurry
    sentiment_dict=sid_obj.polarity_scores(data)
    if sentiment_dict['compound']>= 0.05:
        label="Positive"
    elif sentiment_dict['compound']<= -0.05:
        label="Negative"
    else:
        label="Neutral"
    return {'sentiments':data,'labels':label}

labelledSamples=pd.DataFrame([sentimentLabel(sample) for sample in randomSamples])
labelledSamples[['sentiments','labels']][0:20]

,sentiments,labels
0,Yassssss pleaseeee inject me with that science...,Positive
1,>There is no way to conclusively proof ANY new...,Positive
2,"It is a terrible plan, one fuck up and hospita...",Negative
3,[relevant xkcd](https://xkcd.com/2391/),Neutral
4,I know what you are saying. I trust the scienc...,Negative
5,If only it could save us from the people who d...,Positive
6,What was your personal reaction? As someone wh...,Negative
7,Rushed? Do you think they started this from sc...,Positive
8,They have even been able to do some testing ph...,Positive
9,"Yes, although most of us will probably just go...",Positive


### The samples after being cleaned

In [199]:
labelledSamples['sentiments']=labelledSamples['sentiments'].str.lower()
labelledSamples['sentiments']=[removeStopwords(word_tokenize(sentiment)) for sentiment in labelledSamples['sentiments']]

In [200]:
labelledSamples[['sentiments','labels']]

,sentiments,labels
0,"[yassssss, pleaseeee, inject, science, juice, ...",Positive
1,"[>, way, conclusively, proof, new, legislation...",Positive
2,"[terrible, plan, one, fuck, hospitals, way, be...",Negative
3,"[[, relevant, xkcd, ], (, https, :, //xkcd.com...",Neutral
4,"[know, saying, trust, science, trust, corporat...",Negative
...,...,...
9995,"[comments, like, stupid, really, expect, vacci...",Negative
9996,"[’, matter, companies, daycares, schools, etc,...",Neutral
9997,"[first, ever, reddit, funded, scholarship, let...",Positive
9998,"[well, least, data, need, start, administering...",Positive


In [201]:
labelledSamples['sentiments'][0:10]

0    [yassssss, pleaseeee, inject, science, juice, ...
1    [>, way, conclusively, proof, new, legislation...
2    [terrible, plan, one, fuck, hospitals, way, be...
3    [[, relevant, xkcd, ], (, https, :, //xkcd.com...
4    [know, saying, trust, science, trust, corporat...
5               [could, save, us, people, believe, im]
6    [personal, reaction, someone, allergies, news,...
7    [rushed, think, started, scratch, like, year, ...
8    [even, able, testing, phases, overlapping, tim...
9    [yes, although, us, probably, go, moderna, man...
Name: sentiments, dtype: object